In [ ]:
import pycrs
from pycrs import parser
from fiona.crs import from_epsg
from rasterio.crs import CRS
from rasterio.mask import mask
from shapely.geometry import box
import rasterio
from rasterio.plot import show, show_hist
from rasterstats import zonal_stats
import geopandas as gpd
import os
import numpy as np
import pandas as pd
import fiona

In [ ]:
# the files to be processed
data_dir = r'D:\data'    
rasterfile = os.path.join(r'D:\data\rasterdata')
pdata = gpd.GeoDataFrame.from_file(os.path.join(data_dir,'polygons.gpkg'))

#directory for output tiff
ddir = data_dir

# epsg of the datasets (should be projected)
EPSG = 32610

# output file name
outfile = 'polygons_area.csv'

In [ ]:
def getFeatures(gdf):
    # Function to parse features from GeoDataFrame to rasterio
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [ ]:
import ogr
def fractional_pixel_weights(fsrc, geom):

    gt = fsrc.transform
    xs = np.arange(gt[2], gt[2] +  gt[0]* (1 + fsrc.shape[1]), gt[0])
    ys = np.arange(gt[5], gt[5] +  gt[4]* (1 + fsrc.shape[0]), gt[4])

    # Convert geom into ogr geometry
    geom_ogr = ogr.CreateGeometryFromWkt(geom.to_wkt())
    #geom_ogr = shapely.wkt.loads(geom)

    # Loop through each grid cell, compute the intersecting area
    overlapping_areas = np.empty((len(ys)-1, len(xs)-1))
    for ix in range(len(xs)-1):
        xmin = xs[ix]
        xmax = xs[ix + 1]
        for iy in range(len(ys)-1):
            ymax = ys[iy]
            ymin = ys[iy + 1]

            # Intersecting area
            coords_wkt = "POLYGON ((" + str(xmin) + ' ' + str(ymax) + ', ' + str(xmax) + ' ' + str(ymax) + ', ' + str(xmax) + ' ' + str(ymin) + ', ' + str(xmin) + ' ' + str(ymin) + ', ' + str(xmin) + ' ' + str(ymax) + "))"
            polycell = ogr.CreateGeometryFromWkt(coords_wkt)

            overlapping_areas[iy, ix] = polycell.Intersection(geom_ogr).Area()
            

    # Ratio of overlapped area to pixel area
    #frac_intersected = (overlapping_areas / (abs(gt[0] * gt[4])))
    
    # Area of overlap polygon/pixels - 30m resolution
    frac_intersected = (overlapping_areas / (abs(gt[0] * gt[4]))) * 900

    return frac_intersected

In [ ]:
from osgeo import gdal
from osgeo import gdalnumeric
import sys
import time

A = time.time()
B = []

n = 0

area_totals = pd.DataFrame()
df = pd.DataFrame()

pdata['minx'], pdata['miny'], pdata['maxx'], pdata['maxy'] = pdata.bounds['minx'], pdata.bounds['miny'],pdata.bounds['maxx'],pdata.bounds['maxy']

with rasterio.open(rasterfile) as src:
    for feat in zip(pdata['id'], pdata['geometry']):

        # subset the gdf to process one polygon at a time
        gdf = pdata.iloc[n]
        fileid = gdf['id']
        print n

        # Set the geometry field to the polygon to be analyzed
        geofield = gdf['geometry']

        r = rasterio.mask.mask(src, geofield, crop=True, filled=False)
        
        out_name = 'raster'
        
        # create bounding box to clip raster
        minx, miny, maxx, maxy = gdf['minx'], gdf['miny'], gdf['maxx'], gdf['maxy']
        bbox = box(minx, miny, maxx, maxy)
        geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(EPSG))
        coords = getFeatures(geo)
        
        # clip the raster based on the extent of the polygon
        #print n, coords
        out_img, out_transform = mask(src, coords, crop=True, filled=False)
        
        # update metadata
        out_meta = src.meta.copy()
        crs = fiona.crs.from_epsg(EPSG)
        out_meta.update({"driver": "GTiff", "height": out_img.shape[1],"width": out_img.shape[2],"transform": out_transform,"crs": crs})
        
        # output the tiff
        out_tif = os.path.join(ddir, 'raster.tiff')
        with rasterio.open(out_tif, "w", **out_meta) as dest:
            dest.write(out_img) 
            
            # run the analysis for area cover
            x = fractional_pixel_weights(dest, geofield)
            
            #close the file
            dest.close()
            n = n + 1
        
        # Now iterate through each cell in both arrays and summarize the area for each value per pixel   
        # Open the temp clipped raster and get the array values for each cell
        raster = gdal.Open(out_tif)
        rarray = np.array(raster.GetRasterBand(1).ReadAsArray())
        
        # count the number of cells to loop through to retrieve values
        nums = len(x)
        
        for ynum in range(0,nums):
            for xnum in range(0,nums):
                try:
                    val1 = rarray[xnum,ynum]
                    val1area = x[xnum,ynum]
                    
                    # Append the values to the dataframe
                    df['Value'], df['Area'], df['ID'] = val1, val1area, fileid
                    areatotals = df.append(pd.Series([val1,val1area,fileid], index=df.columns), ignore_index=True)
                    area_totals = area_totals.append(areatotals)
                    
                except IndexError as e:
                    a = 1
        del raster
                
    print area_totals.head()
    B.append(time.time()-A)
    print B

In [ ]:
# Reshape the data to sum area for each value
update_totals = area_totals.groupby(['recID','Value'], as_index=False)[['Area']].sum()
update_totals = update_totals.pivot(index=update_totals.recID, columns='Value')['Area'].fillna(0)
#update_totals = update_totals.rename(columns={1:'Sample_Value_Name'})

In [ ]:
update_totals.to_csv(outfile)